<a href="https://colab.research.google.com/github/AssistMoli/LLaMA-Factory/blob/main/%E5%85%85%E5%88%86%E7%99%BC%E6%8F%AE_Colab_%E8%A8%82%E9%96%B1%E7%9A%84%E5%83%B9%E5%80%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests==2.31.0
!pip install bs4==0.0.2
!pip install openai==0.28.0
!pip install transformers==4.40.0
!pip install fake_useragent==1.5.1
!pip install selenium==4.19.0
!pip install webdriver_manager==4.0.1
!pip install xlsxwriter==3.2.0
!apt-get update
!apt install chromium-chromedriver

In [ ]:
import time
from datetime import datetime

from fake_useragent import UserAgent
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

import requests

from bs4 import BeautifulSoup

# 載入 openai 及 huggging face 套件
import openai
from transformers import pipeline

# 關閉 warning
import warnings
from urllib3.exceptions import InsecureRequestWarning
warnings.filterwarnings("ignore", category=InsecureRequestWarning)

import pandas as pd

# 開啟雲端sheet服務
from oauth2client.service_account import ServiceAccountCredentials
import gspread
scopes = ["https://spreadsheets.google.com/feeds"]
credentials = ServiceAccountCredentials.from_json_keyfile_name("moli-linebot-fsc-penalty-258d250e683e.json",scopes)
sheet_service = gspread.authorize(credentials)

# 開啟雲端資料夾服務
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
scope = ['https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('moli-linebot-fsc-penalty-258d250e683e.json', scope)
drive_service = build('drive', 'v3', credentials=credentials)

from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload

import gspread_dataframe as gd



In [ ]:
def track_execution(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()  # Get the current time before calling the function
        print(f"Arguments passed to {func.__name__}: args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        end_time = time.time()    # Get the current time after calling the function
        elapsed_time = end_time - start_time  # Calculate the elapsed time
        print(f"Execution time for {func.__name__}: {elapsed_time:.4f} seconds")
        return result
    return wrapper

In [ ]:
@track_execution
def extract_chinese_names(ner_results):
    chinese_names = []
    current_name = ''
    prev_index = None
    prev_entity = None
    for result in ner_results:
        if ((prev_index is None) or (result['index'] == prev_index + 1)) and\
            ((prev_entity is None) or (  'E-PERSON' not in  prev_entity    )  )     :
            if "PERSON" in result["entity"]:
                current_name += result["word"]
            else:
                pass
        else:
            if current_name:
                chinese_names.append(current_name)
            if "PERSON" in result["entity"]:
                current_name = result['word']
        prev_index = result['index']
        prev_entity = result['entity']
    if current_name:
        chinese_names.append(current_name)
    chinese_names = list(set(chinese_names))
    return chinese_names


In [ ]:
@track_execution
def query(token, url, government, title):

    # ua = UserAgent()
    # user_agent = ua.random
    # chrome_options = Options()
    # chrome_options.add_argument(f'user-agent={user_agent}')
    # chrome_options.add_argument('--no-sandbox')
    # chrome_options.add_argument('--headless')
    # chrome_options.add_argument('--disable - gpu')
    # chrome_options.add_argument('--disable-dev-shm-usage')
    # #if env == "colab":
    # #driver = webdriver.Chrome(options=chrome_options)
    # #elif env == "vm":
    # #    driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), chrome_options=chrome_options)
    # driver = webdriver.Chrome(options=chrome_options)
    # driver.get(url)
    # driver.implicitly_wait(5)
    # soup = BeautifulSoup(driver.page_source, "html.parser")
    # text = soup.prettify() #driver.page_source
    # driver.quit()

    response = requests.get(url, verify=False)
    time.sleep(5)
    soup = BeautifulSoup(response.text, "html.parser")
    text = soup.prettify()

    # print(text)

    max_len = 10000
    generated_text = ""
    for i in range(int(len(text)/max_len)+1):
        openai.api_key = token
        prompt = "你是一個可以依照使用者提供的網頁HTML內容來回答使用者相關政府機關的現任首長是誰的機器人。如果你不確定，請你不要回答任何人名。"
        query = f"請問，依照以下網頁HTML內容，目前的{government}{title}有誰？\n{text[ i * max_len : (i+1) * max_len  ]}"
        model_id = "gpt-3.5-turbo-0125"
        response = openai.ChatCompletion.create(
        model=model_id,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": query}
        ],
        temperature = 0
        )
        generated_text += response['choices'][0]['message']['content'].strip()

    ner_pipeline = pipeline("ner", model="ckiplab/bert-base-chinese-ner", tokenizer="ckiplab/bert-base-chinese-ner")
    ner_results = ner_pipeline(generated_text)
    chinese_names = extract_chinese_names(ner_results)

    # return generated_text, chinese_names
    return generated_text,  "、".join(chinese_names)


In [ ]:
@track_execution
def extract_ner(row):
    token = 'sk-proj-PtzqVLjRhAJdwHEJTm3GT3BlbkFJ3wn2P4MJwuQCwZseZ8aH'
    url = row['連結']
    governement = row['機關']
    title = row['職稱']
    generated_text, chinese_names = query(token, url, governement, title)
    return chinese_names

In [ ]:
# to-do-list 這邊可能要優化，告訴使用者哪些人名有增減

@track_execution
def compare_ner(row):
    if row.iloc[-3] != row.iloc[-2]:
        return 'v'
    else:
        return ''




In [ ]:
def download_single_file(service, file_id, local_file_path, file_name):
    """
    雲端將資料下載到本地端用
    :param service: 認證用
    :param file_id: 雲端檔案的id 可透過 update_file()取得
    :param download_file_path: 將雲端上的資料下載到本地端的位置
    :param download_file_name: 下載到本地端的名稱 e.g. 'hope/'
    :return:
    """
    from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
    import io
    if file_id is not None:
        request = service.files().get_media(fileId=file_id) # os.path.join(download_file_path, download_drive_service_name)
        fh = io.FileIO( os.path.join(local_file_path, file_name   ), 'wb')
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
        print('File downloaded!')
    else:
        print("File not found!")

def download_folder_file(service, folder_id , file_name, local_file_path):
    """
    :param service:
    :param folder_id:
    :param download_file_path: e.g. 'hope/'
    :return:
    """
    query       = f"parents = '{folder_id}' "
    response    = service.files().list( q = query ).execute()
    files       = response.get("files")
    for file in files:
        if file_name in file['name']:
            download_single_file(service, file['id'], local_file_path, file['name'] )


In [ ]:
def extract_date(filename):
    try:
        return pd.to_datetime(filename.split('.')[0][:10], format='%Y-%m-%d')
    except ValueError:
        return pd.NaT

In [ ]:
# 從雲端下載上次檔案

download_folder_file(drive_service, '1zcGFRK0ZATEDr7Ftm4ECwXaYWioPtQb4', 'pepsi', './')

xlsx_files = [file for file in os.listdir('.') if file.endswith('.xlsx')]
xlsx_files.sort(key=lambda x: extract_date(x), reverse=True)
if len(xlsx_files) > 0:
    latest_xlsx = xlsx_files[0]
    print(f"Latest .xlsx file: {latest_xlsx} !")
    df = pd.read_excel(latest_xlsx)
    print(df)
else:
    print("No .xlsx files found in the current directory!")


In [ ]:
# 依照檔案內連結，找出政治人物名稱，並判斷是否有異動

today_date = datetime.now().date()
today_date_str = today_date.strftime("%Y-%m-%d")

df = df.fillna('')

df[f'{today_date_str}'] = df.apply(extract_ner, axis=1)

df['異動'] = df.pop('異動') # 將'異動'欄位移動到最後一欄
df['異動'] = df.apply(compare_ner, axis=1)






In [ ]:
# 決定相關參數

your_file_name =  today_date_str +"-pepsi.xlsx"
your_column_widths = {'政治性職務範圍':50,   '機關': 25, '職稱': 25,'連結': 50,"2023-06-07":50   , f"{today_date_str}":50}

your_sender = 'assist.moli@gmail.com'
your_gmail_password =  'yvocyisentlrvsrq'

sheet = sheet_service.open_by_key("14pGbpAdXFbQ-cypZ66drOHtp_iR4tbJCkrNwcsnycDw").sheet1
user  = gd.get_as_dataframe(sheet)
your_mail_list   = user['usermail'].tolist()
your_mail_list = ['brownwang@yuanta.com']

your_subject = today_date_str + '-pepsi'
your_body = """
您好：

附檔為次 pepsi ，再懇請確認有打勾的異動部分。
如連結或判斷有誤，請重新填寫後再寄給 brownwang@yuanta.com 。

謝謝您。

祝您有個愉快的一天
Assist.MoLi 茉莉貓小助手 敬上
"""

your_attachments = [your_file_name]





In [ ]:
# 調整格式並寄送郵件

with pd.ExcelWriter(your_file_name, engine='xlsxwriter') as writer:
    df.to_excel(writer, index=False, sheet_name='Sheet1')
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    for col_name, width in your_column_widths.items():
        column_index = df.columns.get_loc(col_name)
        worksheet.set_column(column_index, column_index, width)

import os
import sys
import smtplib
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

sender = your_sender
gmail_password = your_gmail_password
recipients = your_mail_list

outer = MIMEMultipart()
outer['From'] = sender
outer['To'] = ', '.join(recipients)
outer.preamble = 'You will not see this in a MIME-aware mail reader.\n'
outer['Subject'] = your_subject
body = your_body
body = MIMEText(body)
outer.attach(body)
attachments = your_attachments
for file in attachments:
    try:
        with open(file, 'rb') as fp:
            print('open', file)
            msg = MIMEBase('application', "octet-stream")
            msg.set_payload(fp.read())
        encoders.encode_base64(msg)
        msg.add_header('Content-Disposition', 'attachment', filename=os.path.basename(file))
        outer.attach(msg)
    except:
        print("Unable to open one of the attachments. Error: ", sys.exc_info()[0])
        raise
composed = outer.as_string()
try:
    with smtplib.SMTP('smtp.gmail.com', 587) as s:
        s.ehlo()
        s.starttls()
        s.ehlo()
        s.login(sender, gmail_password)
        s.sendmail(sender, recipients, composed)
        s.close()
    print("Email sent!")
except:
    print("Unable to send the email. Error: ", sys.exc_info()[0])
    raise


In [ ]:
# 將資料上傳到特定雲端資料夾

file_metadata = {
    'name': your_file_name,
    'parents': ['1zcGFRK0ZATEDr7Ftm4ECwXaYWioPtQb4'],
    'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
}
media = MediaFileUpload(your_file_name,
                        mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
                        resumable=True)
file = drive_service.files().create(body=file_metadata, media_body=media).execute()
del file

print('File uploaded!')

In [ ]:
# 刪除地端 .xlsx

import os
files = os.listdir('.')
xlsx_files = [file for file in files if file.endswith('.xlsx')]
for file in xlsx_files:
    os.remove(file)
print("All .xlsx files deleted!")